In [1]:
import os
import json
from IPython.display import display, Markdown
from dotenv import load_dotenv
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#base langchain library imports
from langchain_openai import ChatOpenAI
import langgraph
from langgraph.prebuilt import create_react_agent
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

#tableau_langchain imports
from utilities.tableau.readMetadata import read
from utilities.tableau.setPrompt import instantiate_prompt
from tools.tableau.getDataTool import get_data

In [6]:
#read in the environment varibales to query a Tableau Published Data Source from a target Tableau Cloud Site or Server Instance
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [7]:
## Use the read module to pull in the necessary metadata from your target Tableau Published Data Source. This will module also augment the
## base metadata provided by the read_metadata endpoint by removing unneccesary fields and adding sample field values to each string field.
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)

# use the instaniate_prompt module to add the data model metadata to the nlq_to_vds prompt
instantiate_prompt(metadata = metadata)

reading in your field metadata...
looking up field values...
prompt is ready!


{'instructions': "\nYou are an expert at writing API request bodies for Tableau’s HeadlessBI API.\nThe HBI query is a JSON object that contains 2 fundamental components.\n    1. fields [required] - an array of fields that define the desired output of the query\n    2. filters [optional] - an array of filters to apply to the query. They can include fields that are not in the fields array.\nYour task is to retrieve data relevant to a user’s natural language query.\n\nQuery as much data as might be useful; it's ok if you pull in superfluous fields,\nYou will be successful if you bring back all the data that could help to answer the question, even if additional transformation and actions are needed.\n\nYou can find the fieldCaptions by checking the data_model field.\nKeep your output very structured. Use the following structure:\nReasoning:\n\nJSON_payload:\nMake sure you use this structure so that it's simple to parse the output.\nReturn query results verbatim so the pandas agent can anal

In [8]:
#Langchain Example
# Initialize an LLM and an agent with the get_data tool to answer natural language questions with Tableau Published Datasources
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS, # Use OpenAI's function calling
    verbose = False
)

# Run the agent
response = tableauHeadlessAgent.invoke("what sales per region?")
display(Markdown(response['output']))

Here are the sales figures per region:

- **West**: $739,813.61
- **East**: $691,828.17
- **Central**: $503,170.67
- **South**: $391,721.91

In [ ]:
#Langgraph Example
# Initialize an LLM and an agent with the get_data tool to answer natural language questions with Tableau Published Datasources
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

query = "what are sales per region?"

tableauHeadlessAgent = create_react_agent(model, tools)

# Run the agent
messages = tableauHeadlessAgent.invoke({"messages": [("human",'which products are my top customers buying?')]})
display(Markdown(messages['messages'][3].content))